# Notebook to read in brownian simulation output and animate the data

## Create Animation by calling c++ code from bash.
- Modifications to physical parameters are handled in main.cpp

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Get Geometric Info
print 'GEOMETRY PARAMS--------------------------------------------------'
params = pd.read_csv('params.txt',sep='\t')
R = params.R[0]
a = params.a[0]
velo = params.velocity[0]
r_enc = params.r_enc[0]
N_ants = params.num_ants[0]
coll_flag = params.collision_flag[0]
print '\tBox Radius: ',R
print '\tAperture Size: ',a
print '\tNumber of Ants: ',N_ants
print '\tEncounter Radius: ',r_enc
print '\tVelocity: ',velo
if coll_flag == 1:
    print '\tCollisions On.'
else:
    print '\tCollisions Off.'

## Get Simulation Info
print 'SIMULATION PARAMS------------------------------------------------'
data = pd.read_csv('output.txt',sep='\t')
uniq_time = np.unique(data.event_time)
t_max = uniq_time[-1]
t_min = uniq_time[1]-uniq_time[0]
for i in range(np.size(uniq_time)-1):
    delta_t = uniq_time[i+1] - uniq_time[i]
    if delta_t < t_min:
        t_min = delta_t # Store the minimum time between events and use that as the resolution
N_events = data.shape[0]/N_ants
print '\tNumber of Events: ',N_events
print '\tTotal Time: ',t_max
print '\tMinimum Time Between Events: ',t_min

## Animation Parameters
print 'ANIMATION PARAMS-------------------------------------------------'
N_frames = 1000
output_every = 1
color1_ants = [0]  # list of ants to be colored with a different color
color1 = 'r'  # choose a color for unique ants
color2 = 'k'  # choose a color for unique ants
colorz = []  # array to store ant colors for each timestep
plot_dir = '/Users/jakehanson/Documents/Ants/plots/'
output_path = '/Users/jakehanson/Desktop/sim.gif'
dt = t_max/N_frames
print '\tList of Colored Ants: ',color1_ants
print '\tNumber of Frames: ',N_frames
print '\tOutput Every: ',output_every
print '\tOutput Resolution: ',dt*output_every
print '\tPlot Directory: ',plot_dir
print '\tAnimation Output: ',output_path


## Generate Disk for Animation
N_points = 100
x_disk = []
y_disk = []
theta_crit = float(a)/R
for i in range(N_points+1):
    x = R*np.cos((np.pi+theta_crit)/2+(2*np.pi-theta_crit)*i/N_points)
    y = R*np.sin((np.pi+theta_crit)/2+(2*np.pi-theta_crit)*i/N_points)
    x_disk.append(x)
    y_disk.append(y)

    
## GENERATE ANIMATION
print 'RUNNING ANIMATION------------------------------------------------'
!rm $plot_dir/*
time_array = np.linspace(0,t_max,N_frames)  #generate evenly spaced time series
counter = -1


#Since the simulation is event based, we need to smoothly interpolate data through time
for t in time_array:
    counter = counter+1
    
    # get index of data array as close to sample time as possible
    for j in range(np.size(uniq_time)):
        if uniq_time[j]-t <= 0.:
            index = j*N_ants
        else:
            break
            
    # Initialize arrays to hold data for this frame
    x_positions, y_positions = [], []
    
    # Get x,y positions of ants at this time step
    for ant in range(N_ants):
        current_time = data.event_time[index]
        next_time=data.event_time[index+ant]
        if current_time != next_time:
            raise RuntimeError('ERROR: Plotting data from different time steps.')
            
        if data.in_nest[index+ant] == 1:
            interpolate_x = data.x[index+ant]+data.v_x[index+ant]*(t-data.event_time[index+ant])
            interpolate_y = data.y[index+ant]+data.v_y[index+ant]*(t-data.event_time[index+ant])
            
            # See if we went out of bounds
            if interpolate_x**2+interpolate_y**2 > R**2:
                raise RuntimeError('ERROR: Interpolation out of Bounds. Try increasing resolution.')
                break
            else:
                x_positions.append(interpolate_x)
                y_positions.append(interpolate_y)
                if ant in color1_ants:
                    colorz.append(color1)
                else:
                    colorz.append(color2)
                
    ## Plot
    if counter%output_every == 0:
        print '\tCounter {:d}/{:d}'.format(counter,N_frames)
        print '\t\tN =',np.size(x_positions)
        plt.plot(x_disk,y_disk,color='k',linestyle='-',linewidth=2)
        plt.axis('off')
        #plt.title('Time: {:5.4f}'.format(t))
        plt.title('R={:2.1f}, a={:2.1f}, r_encounter={:2.1f}'.format(R,a,r_enc))
        plt.scatter(x_positions,y_positions,s=100,color=colorz,alpha=0.7)
        plt.xlim((-R-r_enc,R+r_enc))
        plt.ylim((-R-r_enc,R+r_enc))
        plt.axes().set_aspect('equal', 'datalim')
        fname = ('plot' + '_%06d.png' % (counter))  # assign filename
        plt.savefig(plot_dir+fname)
        plt.clf()

plt.close()
print 'RUNNING CONVERSION-----------------------------------------------'
!convert $plot_dir/* $output_path
print 'DONE!'

GEOMETRY PARAMS--------------------------------------------------
	Box Radius:  2
	Aperture Size:  1
	Number of Ants:  20
	Encounter Radius:  0.1
	Velocity:  0.1
	Collisions On.
SIMULATION PARAMS------------------------------------------------
	Number of Events:  329
	Total Time:  996.41
	Minimum Time Between Events:  0.00259999999999
ANIMATION PARAMS-------------------------------------------------
	List of Colored Ants:  [0]
	Number of Frames:  1000
	Output Every:  1
	Output Resolution:  0.99641
	Plot Directory:  /Users/jakehanson/Documents/Ants/plots/
	Animation Output:  /Users/jakehanson/Desktop/sim.gif
RUNNING ANIMATION------------------------------------------------
	Counter 0/1000
		N = 20
	Counter 1/1000
		N = 20
	Counter 2/1000
		N = 20
	Counter 3/1000
		N = 20
	Counter 4/1000
		N = 20
	Counter 5/1000
		N = 20
	Counter 6/1000
		N = 20
	Counter 7/1000
		N = 20
	Counter 8/1000
		N = 20
	Counter 9/1000
		N = 20
	Counter 10/1000
		N = 20
	Counter 11/1000
		N = 20
	Counter 12/1000


RuntimeError: ERROR: Interpolation out of Bounds. Try increasing resolution.

In [11]:
help(raise )

SyntaxError: invalid syntax (<ipython-input-11-140984b14e70>, line 1)